In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
#for dirname, _, filenames in os.walk('/kaggle/input/'):
    #for filename in filenames:
       #os.path.join(dirname, filename)

# Any results you write to the current directory are saved as output.

In [14]:
df = pd.read_csv("F:\\4-1\\Thesis\\resized_train_cropped\\trainLabels_cropped.csv", header=None)
df = df.iloc[1:]
num = len(df)
num

35108

In [15]:
data_size = num

In [16]:
train_images_route = "F:\\4-1\\Thesis\\resized_train_cropped\\resized_train_cropped\\"

In [ ]:
def process_image(dfiloc: np.int64, imgloc: str):
    img1 = cv2.imread(imgloc,1)
    img1 = cv2.resize(img1, (350,350))
    processed_image = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    return processed_image

class_list=[]
img=[]

variables_config = {
    '0': (0, 10),
    '1': (0, 2),
    '2': (0, 4)
}

for i in range(data_size):
    imgloc = train_images_route + df.iloc[i,2] + '.jpeg'
    df_location = df.iloc[i,3]
    class_list.append(df_location)

    if df_location not in variables_config:
        img.append(process_image(df_location, imgloc))
        continue
    
    variables_config[df_location][0] += 1
    if variables_config[df_location][0] % variables_config[df_location][1] == 0:
        img.append(process_image(df_location, imgloc))


In [ ]:
from collections import Counter

counted_variables = Counter(class_list)
print(counted_variables)

In [ ]:
new_data_size = len(class_list)
new_data_size

In [ ]:
area_of_exudate=[]
gre = []
for image in img:
    img2 = np.array(image)
    r, greencha, b = cv2.split(img2)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8)) 
    curImg = clahe.apply(greencha)
    gre.append(curImg)
    strEl = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))
    curImg = cv2.dilate(curImg, strEl)
    curImg = cv2.medianBlur(curImg,5)
    retValue, curImg = cv2.threshold(curImg, 235, 255, cv2.THRESH_BINARY)
    #curImg= cv2.cvtColor(curImg,cv2.COLOR_BGR2RGB)

    rv2, optical_disk = cv2.threshold(b, 240, 255, cv2.THRESH_BINARY)
    curImg = cv2.subtract(curImg,optical_disk)
    
    
    moment = cv2.moments(curImg)
    huMoments = cv2.HuMoments(moment)

    #for i in range(0,7):
    #    huMoments[i] = -1 * math.copysign(1.0, huMoments[i]) * np.log10(abs(huMoments[i]))
    #Humoments2 = -np.sign(Humoments)*np.log10(np.abs(Humoments))
    area_of_exudate.append(huMoments[0])
    
area_of_exudate = [y for x in area_of_exudate for y in x]
print(area_of_exudate)

In [ ]:
kernel_for_bv = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

def extract_bv(image):

    contrast_enhanced_green_fundus = image
   
    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
   
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
   
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
   # cv2.imshow('contrast_enhanced_green_fundus',contrast_enhanced_green_fundus)
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)
   # cv2.imshow('f5',f5)
    ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)
    mask = np.ones(f5.shape[:2], dtype="uint8") * 255
    #print(mask)
   # _, contours, _ = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(f6.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]

    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
            
    im = cv2.bitwise_and(f5, f5, mask=mask)
    ret, fin = cv2.threshold(im, 15, 255, cv2.THRESH_BINARY_INV)
    newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)), iterations=1)

   
    fundus_eroded = cv2.bitwise_not(newfin) 
    xmask = np.ones(image.shape[:2], dtype="uint8") * 255
    xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
    blood_vessels = cv2.bitwise_not(finimage)
    return blood_vessels

In [ ]:
area_of_bloodvessel = []
image_size = (350, 350)

for image in gre:
    bloodvessel = extract_bv(image)
    bloodvessel = cv2.resize(bloodvessel, image_size)
    count = 0
    bloodvessel = 255 - bloodvessel
    retValue, bloodvessel = cv2.threshold(bloodvessel, 235, 255, cv2.THRESH_BINARY)
    bloodvessel = cv2.dilate(bloodvessel,kernel_for_bv,iterations=1)
    
    moment = cv2.moments(bloodvessel)
    huMoments = cv2.HuMoments(moment)
 
    area_of_bloodvessel.append(huMoments[0])
    
area_of_bloodvessel = [y for x in area_of_bloodvessel for y in x]
print(area_of_bloodvessel)   

In [ ]:
kernelmicro = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))


In [ ]:
def extract_ma(image):
     
    median = cv2.medianBlur(image,3)

    erosion_ma = 255 - cv2.erode(median, kernelmicro,iterations=1)
    ret3,thresh2 = cv2.threshold(erosion_ma,215,255,cv2.THRESH_BINARY)
    closing_ma = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernelmicro)
    mask = np.ones(closing_ma.shape[:2], dtype="uint8") * 255
    contours_mn, hierarchy_mn = cv2.findContours(closing_ma, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
   
    for cnt_mn in contours_mn:
        if cv2.contourArea(cnt_mn) <= 70:
            cv2.drawContours(mask, [cnt_mn], -1, 0, -1)
            
    final_ma = cv2.bitwise_and(closing_ma, closing_ma, mask=mask)
    sub_ma = cv2.subtract(closing_ma,final_ma)
    sub_ma = cv2.morphologyEx(sub_ma, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
    sub_ma =cv2.erode(sub_ma,kernelmicro,iterations=1)
    return sub_ma


In [ ]:
area_of_micro = []
for image in gre:
    count = 0
    mcran = extract_ma(image)
    
    moment = cv2.moments(mcran)
    huMoments = cv2.HuMoments(moment)
    area_of_micro.append(huMoments[0])

area_of_micro = [y for x in area_of_micro for y in x]
print(area_of_micro)
  

In [ ]:
X = list(zip(area_of_exudate,area_of_bloodvessel, area_of_micro))
y = class_list


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_config = {
    'test_size': .25,
    'random_state': 0
}
X_train, X_test , y_train, y_test = train_test_split(X, y, **train_config)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier_config = {
    'n_estimators': 3000,
    'criterion': 'gini',
    'max_features': 'sqrt',
    'random_state': 1,
    'oob_score': True
}
Classifier = RandomForestClassifier(**classifier_config)
Classifier.fit(X_train, y_train)


In [ ]:
y_pred = Classifier.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)


In [ ]:
from plot_data import plot_using_plt, plot_using_plotly

class_list = pd.to_numeric(class_list)
colors = class_list

x = area_of_bloodvessel
y = area_of_exudate
z = area_of_micro

plot_using_plt(x, y, z, colors)


In [ ]:
x = area_of_bloodvessel
y = area_of_micro
z = area_of_exudate

plot_using_plotly(x, y, z, colors)
